# INF391 - Tarea 9
El objetivo de esta tarea es construir un sistema de recomendación basado en contenido usando procesamiento del lenguaje natural.
- El conjunto de datos son las 250 películas mejor ranqueadas de IMDB.
- Las recomendaciones estarán basadas en información como directores, actores, género y descripción de la película.
- Realizar una limpieza de los datos para considerar solo las palabras más relevantes de la descripción de la película.
- Vectorizar cada película y calcular su similaridad con el resto.
- La entrada será el título de una película y la salida debe ser una lista con las 10 más similares (Top-10).
- ¿Cómo cambian las recomendaciones si están basadas únicamente en el título de la película?
- ¿Alguna otra *feature* del conjunto original sería interesante incluir en el análisis?

In [1]:
import pandas as pd
df = pd.read_csv('IMDB_Top250.csv')

In [2]:
df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [3]:
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [8]:
# Limpieza dataframe
def limpiar_df(df):
    ## A lower case y eliminar puntuación
    ## (Dejar nombres capitalizados)
    df['Title'] = df['Title'].str.lower().str.replace('[,.:]', '', regex=True)
    df['Genre'] = df['Genre'].str.lower().str.replace(',', '', regex=True)
    df['Plot'] = df['Plot'].str.lower().str.replace('[,.:]', '', regex=True)
    df['Actors'] = df['Actors'].str.replace('[,.:]', '', regex=True)
    df['Director'] = df['Director'].str.replace('[,.:]', '', regex=True)
    ## Eliminar stopwords de plot
    sw = stopwords.words('english')
    df['Title'] = df['Title'].apply(lambda x: ' '.join([palabra for palabra in x.split() if palabra not in sw]))
    df['Plot'] = df['Plot'].apply(lambda x: ' '.join([palabra for palabra in x.split() if palabra not in sw]))
    # Agregar columna ID
    df['id'] = df.index
    
# Unir contenido en un string
# lista de features
def componerContenido(df, features):
    df_contenido = pd.DataFrame(index=df.index)
    df_contenido['id'] = df['id']
    # Concatenar
    total = ""
    for feat in features:
        parcial = df[feat]
        total += " "+parcial
    df_contenido['contenido'] = total
    df_contenido['contenido'].str.strip()
    return df_contenido

# Proceso de lemmatización
lemmatizer = WordNetLemmatizer()
lematizar = lambda string: lemmatizer.lemmatize(string)
    
def lemmatizarLista(lista):
    #print("ANTES: ",lista)
    lista=list(map(lematizar,lista))
    #print("DESPUES: ",lista)
    return lista

def lemmatizarContenido(df):
    df['lemma'] = df['contenido'].str.split().apply(lambda x: lemmatizarLista(x))
    
limpiar_df(df)
display(df.head())
features = ['Title', 'Genre', 'Director', 'Actors', 'Plot']
contenido = componerContenido(df, features)
lemmatizarContenido(contenido)
display(contenido.head())
palabras = sum(contenido['lemma'].tolist(), [])
palabras = list(set(palabras))
palabras
frecuencias = np.zeros((len(palabras), df.shape[0]))
i = 0
for pal in palabras:
    for j in range(250):
        freq = contenido['lemma'][j].count(pal)
        frecuencias[i][j] = freq
    i += 1
display(frecuencias)

,Title,Genre,Director,Actors,Plot,id
0,shawshank redemption,crime drama,Frank Darabont,Tim Robbins Morgan Freeman Bob Gunton William ...,two imprisoned men bond number years finding s...,0
1,godfather,crime drama,Francis Ford Coppola,Marlon Brando Al Pacino James Caan Richard S C...,aging patriarch organized crime dynasty transf...,1
2,godfather part ii,crime drama,Francis Ford Coppola,Al Pacino Robert Duvall Diane Keaton Robert De...,early life career vito corleone 1920s new york...,2
3,dark knight,action crime drama,Christopher Nolan,Christian Bale Heath Ledger Aaron Eckhart Mich...,menace known joker emerges mysterious past wre...,3
4,12 angry men,crime drama,Sidney Lumet,Martin Balsam John Fiedler Lee J Cobb EG Marshall,jury holdout attempts prevent miscarriage just...,4


,id,contenido,lemma
0,0,shawshank redemption crime drama Frank Darabo...,"[shawshank, redemption, crime, drama, Frank, D..."
1,1,godfather crime drama Francis Ford Coppola Ma...,"[godfather, crime, drama, Francis, Ford, Coppo..."
2,2,godfather part ii crime drama Francis Ford Co...,"[godfather, part, ii, crime, drama, Francis, F..."
3,3,dark knight action crime drama Christopher No...,"[dark, knight, action, crime, drama, Christoph..."
4,4,12 angry men crime drama Sidney Lumet Martin ...,"[12, angry, men, crime, drama, Sidney, Lumet, ..."


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
#df.shape[0]
#contenido['lemma'][0]
contenido.tail()

,id,contenido,lemma
245,245,lost weekend drama film-noir Billy Wilder Ray...,"[lost, weekend, drama, film-noir, Billy, Wilde..."
246,246,short term 12 drama Destin Daniel Cretton Bri...,"[short, term, 12, drama, Destin, Daniel, Crett..."
247,247,girl friday comedy drama romance Howard Hawks...,"[girl, friday, comedy, drama, romance, Howard,..."
248,248,straight story biography drama David Lynch Si...,"[straight, story, biography, drama, David, Lyn..."
249,249,slumdog millionaire drama Danny Boyle Lovelee...,"[slumdog, millionaire, drama, Danny, Boyle, Lo..."


In [6]:
#stopwords.words('english')

In [7]:
lemmatizer = WordNetLemmatizer()
c=lemmatizer.lemmatize("forces")
print(c)
a = ["the", "forces", "rocks", "years"]
b=list(map(lematizar, a))
b

force


['the', 'force', 'rock', 'year']